# # TP10: Multinomial Naive Bayes

**Alumna**: Lucia Berard

**Fecha**: 15/06/2025

[Link a Google Colab](https://colab.research.google.com/drive/1ah4GT2vGlptvJM9qctPRs0NpqqozR07z?usp=sharing)


La base de datos *BuzzFeed-Webis Fake News Corpus 2016* posee diferentes artículos periodísticos de una semana cercana a las elecciones estadounidenses de ese año. Se desea entrenar un algoritmo Multinomial Naive Bayes capaz de clasificar los artículos en: “mayormente falso”, “mayormente verdadero”, “mezcla de verdadero y falso” y “sin contenido factual”.

![Banner UNICEF Argentina](https://www.unicef.org/argentina/sites/unicef.org.argentina/files/styles/media_banner/public/3%20%282%29.webp?itok=czU8qvE-)

In [34]:
import os
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

print('sklearn:', sklearn.__version__)
print('numpy:', np.__version__)


sklearn: 1.6.1
numpy: 2.0.2


____
## (a) Exploración de datos:

#### a.1) Descargar la base de datos en https://zenodo.org/record/1239675/files/articles.zip?download=1.


In [35]:
!wget -nc 'https://zenodo.org/record/1239675/files/articles.zip?download=1' -O articles.zip
!unzip -n articles.zip

zsh:1: command not found: wget
Archive:  articles.zip



#### a.2) Construir la base de datos. Puede usar el siguiente código:

```python
import xml.etree.ElementTree as ET
data = {"mainText": [], "orientation": [], "veracity": []}

for filename in os.listdir("articles/"):
    root = ET.parse(f"articles/{{filename}}").getroot()
    for elem in root:
        if elem.tag in data.keys():
                data[elem.tag].append(elem.text)

    data = pd.DataFrame(data)
    data = data[data.notna().all(axis="columns")]
```


In [36]:
data = {"mainText": [], "orientation": [], "veracity": []}

for filename in os.listdir("articles/"):
    root = ET.parse(f"articles/{filename}").getroot()
    for elem in root:
        if elem.tag in data.keys():
            data[elem.tag].append(elem.text)

# Convertir a DataFrame y eliminar filas con valores nulos
df = pd.DataFrame(data)
df = df[df.notna().all(axis="columns")]

print("Primeras filas del DataFrame:")
display(df.head())

Primeras filas del DataFrame:


,mainText,orientation,veracity
0,Millions of people tuned in Monday night to wa...,left,mostly true
1,The Clintons understand the average American. ...,right,mixture of true and false
2,Harassment is known in Arabic as ‘taharrush’. ...,right,mixture of true and false
3,Democratic President Barack Obama pulled off a...,left,mostly true
4,Democratic nominee Hillary Clinton knows her f...,left,mostly true



#### a.3) Utilice el comando `train_test_split` (sklearn) para definir dos conjuntos de datos. El conjunto de entrenamiento debe contener el 80% de las muestras, el resto será de testeo.

Para definir el 80% de las muestras, utilice el parámetro `test_size=0.2` que indica el porcentaje de muestras que se utilizarán para el testeo.

#### a.4) Utilizando `CountVectorizer` (sklearn) pre-procesar los datos del texto principal de los artículos. Se recomienda convertir el texto a minúscula, utilizar como `stop_words` las palabras estándar del idioma inglés, eliminar las palabras que aparecen en más del 60% de los documentos y descartar las palabras vistas en menos de 3 documentos.

[`CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) de `sklearn` convierte una colección de documentos de texto en una matriz de conteo de palabras. Esto significa que toma un conjunto de textos y los transforma en una matriz, donde cada columna representa una palabra distinta del vocabulario y cada fila representa un documento. El valor en cada celda es la cantidad de veces que aparece esa palabra en ese documento.

Por ejemplo, si tengo tres textos: "hola mundo", "hola" y "mundo hola hola"

El `CountVectorizer` generará una matriz como esta:

| | hola | mundo |
|------|------|-------|
| doc1 | 1 | 1 | 
| doc2 | 1 | 0 |
| doc3 | 2 | 1 |

In [37]:
# Separar en conjuntos de entrenamiento y testeo (80%/20%)
X = df["mainText"]
y = df["veracity"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# test_size=0.2: 20% de los datos para testeo
# random_state=42: el numero es arbitrario, el objetivo es que el resultado sea reproducible

# Preprocesamiento con CountVectorizer
vectorizer = CountVectorizer(
    lowercase=True,
    stop_words='english', # Elijo ingles como idioma
    max_df=0.6,    # Elimino palabras que aparecen en más del 60% de los documentos
    min_df=3       # Eliminp palabras que aparecen en menos de 3 documentos
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

n_total = len(df)
n_train = X_train_vec.shape[0]
n_test = X_test_vec.shape[0]

pct_train = n_train / n_total * 100
pct_test = n_test / n_total * 100

print(f"Forma del set de entrenamiento: {X_train_vec.shape} "
      f"\n(Significa que tiene {n_train} documentos, {pct_train:.1f}% del total)")
print(f"Forma del set de testeo: {X_test_vec.shape} "
      f"\n(Significa que tiene {n_test} documentos, {pct_test:.1f}% del total)")

# Funcion para mostrar la distribución de clases
def show_distribution(df, column, name=None):
    """
    Muestra la distribución de clases de una columna de un DataFrame.
    """
    counts = df[column].value_counts().sort_index()
    percent = df[column].value_counts(normalize=True).sort_index() * 100
    distribucion = pd.DataFrame({
        "Cantidad": counts,
        "Porcentaje": percent.round(2)
    })
    if name is None:
        name = column
    print(f"Distribución de clases para '{name}':")
    display(distribucion)

show_distribution(df, "veracity")
show_distribution(df, "orientation")


Forma del set de entrenamiento: (1283, 11191) 
(Significa que tiene 1283 documentos, 80.0% del total)
Forma del set de testeo: (321, 11191) 
(Significa que tiene 321 documentos, 20.0% del total)
Distribución de clases para 'veracity':


,Cantidad,Porcentaje
veracity,,
mixture of true and false,209,13.03
mostly false,82,5.11
mostly true,1249,77.87
no factual content,64,3.99


Distribución de clases para 'orientation':


,Cantidad,Porcentaje
orientation,,
left,252,15.71
mainstream,822,51.25
right,530,33.04


Se verifica que el set de entrenamiento tiene 1283 documentos y el de testeo 321 documentos, de un total de 1604 documentos, por lo que cumple el ratio de 80/20. 

Además, se analizó la distribución de las clases de veracidad y orientación. Por ejemplo, la mayoría de los textos (77.87%) son "mostly true" y la mayoría de las orientaciones (51.25%) son "mainstream".

Se nota que los valores para veracidad son más desiguales que los para orientación, lo cual podría indicar que hay un sesgo en el dataset y que los datos pueden generar problemas en los siguientes resultados.




## (b) Entrenamiento:

Implementar un MNB de $\alpha = (1, 1, \cdots, 1)$ que prediga la veracidad de un artículo a partir de su texto principal (pre-procesado). El código debe estar estructurado de la siguiente manera:

```python
class MNB:
    # Inicializar atributos y declarar hiperparámetros
    def __init__(self,...):
        ...

    # Etapa de entrenamiento
    def fit(self,X,y):
        ...

    # Etapa de testeo soft
    def predict_proba(self,X):
        ...

    # Etapa de testeo hard (no repetir código)
    def predict(self,X):
        ...
```



`MNB: Multinomial Naive Bayes`

El modelo Multinomial Naive Bayes asume que las palabras de un documento son independientes entre sí; es decir, la aparición de una palabra no afecta la probabilidad de aparición de otra (de ahí el término "Naive"). El objetivo es modelar la distribución multinomial de las palabras en los diferentes documentos del dataset.

**Teorema de Bayes**

El teorema de Bayes nos dice que:

$$ P(\text{clase} \mid \text{documento}) = \frac{P(\text{documento} \mid \text{clase}) \times P(\text{clase})}{P(\text{documento})} $$

Para clasificar, buscamos la clase que maximiza esta probabilidad:

$$ \text{clase}_{\text{predicha}} = \arg\max{\left[P(\text{clase} \mid \text{documento})\right]} $$

Como $P(\text{documento})$ es constante para todas las clases, se puede omitir en la maximización:

$$ \text{clase}_{\text{predicha}} = \arg\max{\left[P(\text{documento} \mid \text{clase}) \times P(\text{clase})\right]} $$

Aplicando logaritmos (para mayor estabilidad numérica):

$$ \text{clase}_{\text{predicha}} = \arg\max{\left[ \log P(\text{clase}) + \log P(\text{documento} \mid \text{clase}) \right]} $$

En el caso del modelo multinomial, la verosimilitud del documento dada la clase se calcula como:

$$ \log P(\text{documento} \mid \text{clase}) = \sum_{i} \text{count}(\text{palabra}_i) \times \log P(\text{palabra}_i \mid \text{clase}) $$

Para realizar el MNB, se siguen los siguientes pasos:

1. fit(X, y): Aprende las probabilidades a partir de los datos de entrenamiento:
    - Calcula la probabilidad de cada clase (P(clase), frecuencia de cada clase).
    - Calcula la probabilidad de cada palabra dado cada clase (P(palabra|clase)) usando suavizado de Laplace:
    $$ P(\text{palabra}_i | \text{clase}) = \frac{\text{count}(\text{palabra}_i, \text{clase}) + \alpha}{\sum_j \text{count}(\text{palabra}_j, \text{clase}) + V \cdot \alpha} $$ 
    donde $V$ es el tamaño del vocabulario.

2. predict_proba(X): Para cada documento, calcula la probabilidad (o log-probabilidad) de que pertenezca a cada clase usando la fórmula de Bayes. Se suele trabajar en log-probabilidades para evitar underflow numérico:
$$ \log P(\text{clase}) + \sum_{i} \text{count}(\text{palabra}_i) \cdot \log P(\text{palabra}_i | \text{clase}) $$

3. predict(X): Devuelve la clase con mayor probabilidad para cada documento (usa los resultados de predict_proba).




## (c) Inferencia

Implementar un método a la clase anterior que calcule el *accuracy* y la *Macro-F1*. Evaluar dichas métricas en el conjunto de testeo. ¿Por qué dan tan diferentes? Para el cálculo de la F1 debe considerar el caso de *precision* y *recall* nulas.

**Métricas**

- Accuracy: Proporción de documentos correctamente clasificados.
- Macro-F1: F1-score promedio por clase, sin ponderar por cantidad de ejemplos por clase.

Para cada clase, calcula:

- Precision = $\frac{\text{True Positives}}{\text{True Positives} + \text{False Positives}}$ Mide la calidad de las predicciones positivas 
- Recall = $\frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}$ Mide la capacidad del modelo para encontrar todos los positivos reales 
- F1 = $\frac{2 \cdot \text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$ Combina ambas métricas en un solo valor, es un promedio balanceado de ambas. Si Precision o Recall son nulos, el F1 para esa clase es 0. 



**¿Por qué pueden diferir tanto accuracy y Macro-F1?**
- Accuracy mide el porcentaje total de aciertos, pero puede estar sesgado si hay clases desbalanceadas (por ejemplo, si una clase es muy frecuente).
- Macro-F1 promedia el F1 de cada clase, dándole igual peso a todas, incluso a las minoritarias. Si el modelo ignora clases poco frecuentes, la Macro-F1 será baja aunque el accuracy sea alto.

In [38]:
class MNB:
    def __init__(self, alpha=1.0):
        self.alpha = alpha
        self.class_log_prior_ = None
        self.feature_log_prob_ = None
        self.classes_ = None

    def fit(self, X, y):
        y = np.asarray(y)
        self.classes_, class_counts = np.unique(y, return_counts=True)
        n_classes = len(self.classes_)
        n_features = X.shape[1]
        self.n_features_ = n_features

        self.class_log_prior_ = np.log(class_counts) - np.log(class_counts.sum())

        feature_count = np.zeros((n_classes, n_features), dtype=np.float64)
        for idx, c in enumerate(self.classes_):
            feature_count[idx, :] = X[y == c].sum(axis=0)

        smoothed_fc = feature_count + self.alpha
        smoothed_cc = smoothed_fc.sum(axis=1, keepdims=True)
        self.feature_log_prob_ = np.log(smoothed_fc) - np.log(smoothed_cc)

    def predict_proba(self, X):
        jll = self._joint_log_likelihood(X)
        return jll

    def predict(self, X):
        jll = self.predict_proba(X)
        idx = np.argmax(jll, axis=1)
        return self.classes_[idx]

    def _joint_log_likelihood(self, X):
        return (X @ self.feature_log_prob_.T) + self.class_log_prior_

    def accuracy(self, X, y):
        y_pred = self.predict(X)
        return np.mean(y_pred == y)

    def macro_f1(self, X, y):
        _, _, f1s = self.per_class_metrics(X, y)
        return np.mean(f1s)

    def per_class_metrics(self, X, y):
        """
        Devuelve arrays de precision, recall y F1 para cada clase.
        """
        y_true = np.asarray(y)
        y_pred = self.predict(X)
        precisions = []
        recalls = []
        f1_scores = []
        for c in self.classes_:
            tp = np.sum((y_pred == c) & (y_true == c))
            fp = np.sum((y_pred == c) & (y_true != c))
            fn = np.sum((y_pred != c) & (y_true == c))
            if tp + fp == 0:
                precision = 0.0
            else:
                precision = tp / (tp + fp)
            if tp + fn == 0:
                recall = 0.0
            else:
                recall = tp / (tp + fn)
            if precision + recall == 0:
                f1 = 0.0
            else:
                f1 = 2 * precision * recall / (precision + recall)
            precisions.append(precision)
            recalls.append(recall)
            f1_scores.append(f1)
        return np.array(precisions), np.array(recalls), np.array(f1_scores)

    def classification_report(self, X, y):
        """
        Devuelve un diccionario con accuracy, macro_f1, y arrays de precisión, recall y F1 por clase.
        """
        acc = self.accuracy(X, y)
        macro_f1 = self.macro_f1(X, y)
        precisions, recalls, f1_scores = self.per_class_metrics(X, y)
        return {
            "accuracy": acc,
            "macro_f1": macro_f1,
            "precisions": precisions,
            "recalls": recalls,
            "f1_scores": f1_scores,
            "classes": self.classes_
        }

def ensure_dense(X):
    if hasattr(X, "toarray"):
        return X.toarray()
    return np.array(X)

mnb = MNB(alpha=1)
X_train_dense = ensure_dense(X_train_vec)
X_test_dense = ensure_dense(X_test_vec)
mnb.fit(X_train_dense, y_train)

report = mnb.classification_report(X_test_dense, y_test)
print("Accuracy:", report["accuracy"])
print("Macro-F1:", report["macro_f1"])
print("\nMétricas por clase:")
for i, class_label in enumerate(report["classes"]):
    print(f"{class_label}:")
    print(f"  Precision: {report['precisions'][i]:.4f}")
    print(f"  Recall: {report['recalls'][i]:.4f}")
    print(f"  F1-score: {report['f1_scores'][i]:.4f}")

Accuracy: 0.6791277258566978
Macro-F1: 0.3717473328129066

Métricas por clase:
mixture of true and false:
  Precision: 0.2297
  Recall: 0.3542
  F1-score: 0.2787
mostly false:
  Precision: 1.0000
  Recall: 0.0588
  F1-score: 0.1111
mostly true:
  Precision: 0.8182
  Recall: 0.8049
  F1-score: 0.8115
no factual content:
  Precision: 0.5000
  Recall: 0.2000
  F1-score: 0.2857


📊 **Análisis de los resultados:**

**Accuracy: 67.9% vs Macro-F1: 37.2%**: La gran diferencia se debe al desbalance de clases y al rendimiento desigual por clase

**Métricas por clase**

- **mostly true**:  ⭐ Clase dominante y bien predicha
  - Precision: 0.82 (de los que predijo como esta clase, 82% eran correctos)
  - Recall: 0.80 (de los reales de esta clase, 80% fueron encontrados)
  - F1: 0.81 (excelente, el modelo predice bien esta clase)

- **mostly false**: 🚨 Problema crítico
  - Precision: 1.00 (cuando predice esta clase, nunca se equivoca)
  - Recall: 0.06 (¡pero casi nunca la predice! Solo 6% de los reales fueron encontrados)
  - F1: 0.11 (muy bajo)

- **no factual content**: ⚠️ Rendimiento medio
  - Precision: 0.50
  - Recall: 0.20
  - F1: 0.29 (mejor que dar un valor random, pero bajo)

- **mixture of true and false**: ⚠️ Rendimiento medio / bajo
  - Precision: 0.23 
  - Recall: 0.35 
  - F1: 0.28 (bajo)


**¿Por qué tanta diferencia entre Accuracy y Macro-F1?**

Accuracy está dominado por la clase "mostly true", que el modelo predice muy bien y probablemente es la clase mayoritaria.
Macro-F1 penaliza fuertemente el mal desempeño en las clases minoritarias ("mostly false", "mixture of true and false", "no factual content").

 Aunque el accuracy global sea alto, si el modelo ignora o predice mal las clases poco frecuentes, Macro-F1 baja mucho.

**Conclusión:**
El modelo parece ser bueno para la clase mayoritaria, pero malo detectando las minoritarias.


## (d) Orientación:

Repetir el ejercicio pero para clasificar la orientación política del portal donde fue publicada la noticia (izquierda, derecha o mainstream) a partir del texto principal preprocesado. ¿Siguen siendo válidas las conclusiones extraídas anteriormente? Justificar.


In [39]:
# Cambia la variable objetivo
X = df["mainText"]
y = df["orientation"]

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorización
vectorizer = CountVectorizer(
    lowercase=True,
    stop_words='english',
    max_df=0.6,
    min_df=3
)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

X_train_dense = ensure_dense(X_train_vec)
X_test_dense = ensure_dense(X_test_vec)

mnb = MNB(alpha=1)
mnb.fit(X_train_dense, y_train)
report = mnb.classification_report(X_test_dense, y_test)

print("Accuracy:", report["accuracy"])
print("Macro-F1:", report["macro_f1"])
print("\nMétricas por clase:")
for i, class_label in enumerate(report["classes"]):
    print(f"{class_label}:")
    print(f"  Precision: {report['precisions'][i]:.4f}")
    print(f"  Recall: {report['recalls'][i]:.4f}")
    print(f"  F1-score: {report['f1_scores'][i]:.4f}")

Accuracy: 0.822429906542056
Macro-F1: 0.7791249491249491

Métricas por clase:
left:
  Precision: 0.6735
  Recall: 0.5893
  F1-score: 0.6286
mainstream:
  Precision: 0.8688
  Recall: 0.9145
  F1-score: 0.8910
right:
  Precision: 0.8214
  Recall: 0.8142
  F1-score: 0.8178


📊 **Análisis de los resultados:**

- Accuracy (82%): Clasifica correctamente el 82%. Es un valor alto, indicando buen rendimiento general.
- Macro-F1 (78%): Tiene mejor rendimiento en ambas clases, no solo en las mayoritarias. Ademas, se parece al valor del accuracy.
- Por clase:
  - mainstream: ⭐ Mejor clase. El modelo es muy bueno (F1: 0.89, precision y recall altos), lo que suele ser esperable si es la clase mayoritaria.
  - right: ✅ Desempeño parecido (F1: 0.82).
  - left: ⚠️ El desempeño es menor (F1: 0.63), pero sigue siendo aceptable. Posiblemente menos ejemplos o más difícil de distinguir

**Comparación de resultados**
- Problema anterior (veracidad):
  - Accuracy: 67.9% vs Macro-F1: 37.2% (diferencia de 30.7%)
  - Rendimiento muy desigual entre clases
- Problema actual (orientación política):
  - Accuracy: 82.2% vs Macro-F1: 77.9% (diferencia de solo 4.3%)
  - Rendimiento mucho más equilibrado

**¿Por qué funciona mejor?**
Porque las clases están más balanceadas. La orientación política parece tener una distribución más equilibrada, lo cual coincide con lo visto en el punto a). Esto puede ser porque hay menos ambigüedad, la orientación política puede ser más fácil de detectar que la veracidad de las noticias.

**Observaciones importantes**:
- Macro-F1 cercano a Accuracy: Indica que el modelo funciona bien en todas las clases
- Rendimiento consistente: Ninguna clase está siendo completamente ignorada. (en veracidad, "mostly false" nunca la detectaba)
- Mejora significativa: De 37.2% a 77.9% en Macro-F1 (¡más del doble!)
